In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop

!whoami
!date

ndbs
Tue Mar 31 11:52:35 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

# Check deaths, YLLs, and YLDs in count data

## 1. Load all count space data tables and see what they are

In [3]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/v4.0_folic_acid_fortification'

locations_rundates = {
    'Ethiopia': '2020_03_18_23_04_26',
    'India': '2020_03_18_23_04_36',
    'Nigeria': '2020_03_18_23_04_42',
}

In [4]:
data = vol.load_all_transformed_count_data(base_directory, locations_rundates)

In [5]:
[k for k in data.keys() if k[0] == 'nigeria']

[('nigeria', 'gestational_age'),
 ('nigeria', 'transition_count'),
 ('nigeria', 'deaths'),
 ('nigeria', 'state_person_time'),
 ('nigeria', 'births_with_ntd'),
 ('nigeria', 'population'),
 ('nigeria', 'person_time'),
 ('nigeria', 'ylls'),
 ('nigeria', 'ylds'),
 ('nigeria', 'births'),
 ('nigeria', 'birth_weight')]

## 2. Load the death, yll, yld tables

In [6]:
deaths = data[('nigeria', 'deaths')]
deaths.head()

,year,age_group,sex,cause,fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,false,death,21,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,false,death,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,false,death,29,baseline,0.0
3,2020,1_to_4,female,diarrheal_diseases,false,death,29,folic_acid_fortification_scale_up,0.0
4,2020,1_to_4,female,diarrheal_diseases,false,death,55,baseline,0.0


In [7]:
deaths.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [8]:
ylls = data[('nigeria', 'ylls')]
ylls.head()

,year,age_group,sex,cause,fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,false,ylls,21,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,false,ylls,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,false,ylls,29,baseline,0.0
3,2020,1_to_4,female,diarrheal_diseases,false,ylls,29,folic_acid_fortification_scale_up,0.0
4,2020,1_to_4,female,diarrheal_diseases,false,ylls,55,baseline,0.0


In [9]:
ylls.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [10]:
ylds = data[('nigeria', 'ylds')]
ylds.head()

,year,age_group,sex,cause,fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,false,ylds,21,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,false,ylds,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,false,ylds,29,baseline,0.0
3,2020,1_to_4,female,diarrheal_diseases,false,ylds,29,folic_acid_fortification_scale_up,0.0
4,2020,1_to_4,female,diarrheal_diseases,false,ylds,55,baseline,0.0


In [11]:
ylds.cause.unique()

array(['diarrheal_diseases', 'iron_deficiency',
       'lower_respiratory_infections', 'measles', 'neural_tube_defects',
       'vitamin_a_deficiency'], dtype=object)

## 3. Check the tables since many rows seem to be 0

I think the zero rows are due to stratification, particularly by coverage.

In [12]:
# This is the typical groupby we'd want:
# groupby_cols = ['year', 'age_group']

def summary_for_cause(df, cause, groupby_cols):
    cause_data = df.query('cause == @cause')
    
    index_cols = ['input_draw','scenario'] + groupby_cols
    
    cause_data = cause_data.groupby(index_cols).value.sum().reset_index()
    cause_data = cause_data.groupby(['scenario']+groupby_cols).value.describe()
    
    return cause_data

### Check diarrheal diseases

In [13]:
groupby_cols = ['year', 'age_group']
summary = summary_for_cause(deaths, 'diarrheal_diseases', groupby_cols)
# summary.head()
# summary.groupby(['scenario']+groupby_cols).value.describe()
summary

count    mean  \
scenario                          year age_group                       
baseline                          2020 1_to_4           25.0  293.00   
                                       early_neonatal   25.0    9.40   
                                       late_neonatal    25.0   18.32   
                                       post_neonatal    25.0  316.00   
                                  2021 1_to_4           25.0  596.32   
                                       early_neonatal   25.0   19.88   
                                       late_neonatal    25.0   40.40   
                                       post_neonatal    25.0  644.64   
                                  2022 1_to_4           25.0  605.76   
                                       early_neonatal   25.0   19.76   
                                       late_neonatal    25.0   36.96   
                                       post_neonatal    25.0  644.24   
                                  2023 1_to_4           25.0  615.96   
                                       early_neonatal   25.0   19.20   
                                       late_neonatal    25.0   36.76   
                                       post_neonatal    25.0  650.64   
                                  2024 1_to_4           25.0  618.36   
                                       early_neonatal   25.0   19.56   
                                       late_neonatal    25.0   36.96   
                                       post_neonatal    25.0  658.20   
folic_acid_fortification_scale_up 2020 1_to_4           25.0  293.00   
                                       early_neonatal   25.0    9.40   
                                       late_neonatal    25.0   18.32   
                                       post_neonatal    25.0  316.00   
                                  2021 1_to_4           25.0  596.32   
                                       early_neonatal   25.0   19.88   
                                       late_neonatal    25.0   40.44   
                                       post_neonatal    25.0  644.64   
                                  2022 1_to_4           25.0  605.72   
                                       early_neonatal   25.0   19.76   
                                       late_neonatal    25.0   36.96   
                                       post_neonatal    25.0  644.28   
                                  2023 1_to_4           25.0  616.00   
                                       early_neonatal   25.0   19.20   
                                       late_neonatal    25.0   36.80   
                                       post_neonatal    25.0  650.56   
                                  2024 1_to_4           25.0  618.40   
                                       early_neonatal   25.0   19.56   
                                       late_neonatal    25.0   36.96   
                                       post_neonatal    25.0  658.20   

                                                              std    min  \
scenario                          year age_group                           
baseline                          2020 1_to_4           57.789128  195.0   
                                       early_neonatal    3.068659    5.0   
                                       late_neonatal     6.853953    8.0   
                                       post_neonatal    55.631975  212.0   
                                  2021 1_to_4          104.066373  430.0   
                                       early_neonatal    4.475116   14.0   
                                       late_neonatal    11.937336   25.0   
                                       post_neonatal   108.989250  460.0   
                                  2022 1_to_4          119.242847  396.0   
                                       early_neonatal    7.281712    5.0   
                                       late_neonatal    10.748953   21.0   
                                       pos

In [14]:
summary_for_cause(ylds, 'diarrheal_diseases', groupby_cols)

count         mean  \
scenario                          year age_group                            
baseline                          2020 1_to_4           25.0   524.563184   
                                       early_neonatal   25.0     1.731685   
                                       late_neonatal    25.0    11.395741   
                                       post_neonatal    25.0   195.953981   
                                  2021 1_to_4           25.0  1060.187979   
                                       early_neonatal   25.0     3.451898   
                                       late_neonatal    25.0    22.859792   
                                       post_neonatal    25.0   397.087003   
                                  2022 1_to_4           25.0  1080.377369   
                                       early_neonatal   25.0     3.455020   
                                       late_neonatal    25.0    22.833076   
                                       post_neonatal    25.0   397.778991   
                                  2023 1_to_4           25.0  1096.592949   
                                       early_neonatal   25.0     3.450295   
                                       late_neonatal    25.0    22.997668   
                                       post_neonatal    25.0   397.392058   
                                  2024 1_to_4           25.0  1109.799425   
                                       early_neonatal   25.0     3.441130   
                                       late_neonatal    25.0    22.957024   
                                       post_neonatal    25.0   399.025777   
folic_acid_fortification_scale_up 2020 1_to_4           25.0   524.563184   
                                       early_neonatal   25.0     1.731685   
                                       late_neonatal    25.0    11.395884   
                                       post_neonatal    25.0   195.954936   
                                  2021 1_to_4           25.0  1060.189447   
                                       early_neonatal   25.0     3.451898   
                                       late_neonatal    25.0    22.860273   
                                       post_neonatal    25.0   397.095893   
                                  2022 1_to_4           25.0  1080.389952   
                                       early_neonatal   25.0     3.455020   
                                       late_neonatal    25.0    22.833332   
                                       post_neonatal    25.0   397.790057   
                                  2023 1_to_4           25.0  1096.619456   
                                       early_neonatal   25.0     3.450340   
                                       late_neonatal    25.0    22.997825   
                                       post_neonatal    25.0   397.399905   
                                  2024 1_to_4           25.0  1109.838587   
                                       early_neonatal   25.0     3.441130   
                                       late_neonatal    25.0    22.957487   
                                       post_neonatal    25.0   399.039556   

                                                              std         min  \
scenario                          year age_group                                
baseline                          2020 1_to_4          106.979294  314.415391   
                                       early_neonatal    0.348613    1.203632   
                                       late_neonatal     2.206952    7.805664   
                                       post_neonatal    41.017866  107.065815   
                                  2021 1_to_4          215.142487  634.428523   
                                       early_neonatal    0.674878    2.261821   
                                       late_neonatal     4.283543   15.920998   
                                       post_neonatal    83.155083  218.380860   
          

### Check neural tube defects

In [15]:
summary_for_cause(deaths, 'neural_tube_defects', groupby_cols)

count   mean  \
scenario                          year age_group                      
baseline                          2020 1_to_4           25.0  12.36   
                                       early_neonatal   25.0   9.36   
                                       late_neonatal    25.0   2.16   
                                       post_neonatal    25.0  10.88   
                                  2021 1_to_4           25.0  23.08   
                                       early_neonatal   25.0  17.20   
                                       late_neonatal    25.0   5.00   
                                       post_neonatal    25.0  25.72   
                                  2022 1_to_4           25.0  26.44   
                                       early_neonatal   25.0  19.40   
                                       late_neonatal    25.0   4.28   
                                       post_neonatal    25.0  25.80   
                                  2023 1_to_4           25.0  33.48   
                                       early_neonatal   25.0  19.84   
                                       late_neonatal    25.0   5.08   
                                       post_neonatal    25.0  28.04   
                                  2024 1_to_4           25.0  38.20   
                                       early_neonatal   25.0  19.04   
                                       late_neonatal    25.0   4.80   
                                       post_neonatal    25.0  27.64   
folic_acid_fortification_scale_up 2020 1_to_4           25.0  12.36   
                                       early_neonatal   25.0   8.80   
                                       late_neonatal    25.0   2.04   
                                       post_neonatal    25.0  10.68   
                                  2021 1_to_4           25.0  22.92   
                                       early_neonatal   25.0  15.64   
                                       late_neonatal    25.0   4.56   
                                       post_neonatal    25.0  23.92   
                                  2022 1_to_4           25.0  25.96   
                                       early_neonatal   25.0  18.32   
                                       late_neonatal    25.0   4.04   
                                       post_neonatal    25.0  23.96   
                                  2023 1_to_4           25.0  31.96   
                                       early_neonatal   25.0  18.44   
                                       late_neonatal    25.0   4.76   
                                       post_neonatal    25.0  26.08   
                                  2024 1_to_4           25.0  35.60   
                                       early_neonatal   25.0  17.28   
                                       late_neonatal    25.0   4.52   
                                       post_neonatal    25.0  25.84   

                                                             std   min   25%  \
scenario                          year age_group                               
baseline                          2020 1_to_4           6.556676   2.0   8.0   
                                       early_neonatal   3.569314   4.0   6.0   
                                       late_neonatal    1.312758   0.0   2.0   
                                       post_neonatal    4.502962   1.0   9.0   
                                  2021 1_to_4           9.349332   9.0  16.0   
                                       early_neonatal   6.075909   7.0  14.0   
                                       late_neonatal    2.843120   0.0   3.0   
                                       post_neonatal   10.506030   9.0  19.0   
                                  2022 1_to_4           9.772751  10.0  18.0   
                                       early_neonatal   4.813176  10.0  16.0   
                                       late_neonatal    3.129430   0.0   2.0   
                                

In [16]:
summary_for_cause(ylls, 'neural_tube_defects', groupby_cols)

count         mean  \
scenario                          year age_group                            
baseline                          2020 1_to_4           25.0  1051.056855   
                                       early_neonatal   25.0   822.585755   
                                       late_neonatal    25.0   189.754407   
                                       post_neonatal    25.0   951.695163   
                                  2021 1_to_4           25.0  1966.761840   
                                       early_neonatal   25.0  1511.577254   
                                       late_neonatal    25.0   439.250471   
                                       post_neonatal    25.0  2249.161368   
                                  2022 1_to_4           25.0  2258.460765   
                                       early_neonatal   25.0  1704.924766   
                                       late_neonatal    25.0   375.990567   
                                       post_neonatal    25.0  2255.448571   
                                  2023 1_to_4           25.0  2861.217414   
                                       early_neonatal   25.0  1743.594901   
                                       late_neonatal    25.0   446.266580   
                                       post_neonatal    25.0  2451.755173   
                                  2024 1_to_4           25.0  3258.021853   
                                       early_neonatal   25.0  1673.290070   
                                       late_neonatal    25.0   421.666970   
                                       post_neonatal    25.0  2416.707665   
folic_acid_fortification_scale_up 2020 1_to_4           25.0  1051.056855   
                                       early_neonatal   25.0   773.371424   
                                       late_neonatal    25.0   179.211968   
                                       post_neonatal    25.0   934.157702   
                                  2021 1_to_4           25.0  1952.871661   
                                       early_neonatal   25.0  1374.478808   
                                       late_neonatal    25.0   400.592285   
                                       post_neonatal    25.0  2091.728038   
                                  2022 1_to_4           25.0  2216.998353   
                                       early_neonatal   25.0  1610.010485   
                                       late_neonatal    25.0   354.908150   
                                       post_neonatal    25.0  2094.630897   
                                  2023 1_to_4           25.0  2730.524562   
                                       early_neonatal   25.0  1620.557844   
                                       late_neonatal    25.0   418.151887   
                                       post_neonatal    25.0  2280.333909   
                                  2024 1_to_4           25.0  3036.033050   
                                       early_neonatal   25.0  1518.614801   
                                       late_neonatal    25.0   397.067712   
                                       post_neonatal    25.0  2259.423350   

                                                               std  \
scenario                          year age_group                     
baseline                          2020 1_to_4           557.427150   
                                       early_neonatal   313.683079   
                                       late_neonatal    115.321005   
                                       post_neonatal    393.721297   
                                  2021 1_to_4           796.844634   
                                       early_neonatal   533.967496   
                                       late_neonatal    249.759118   
                                       post_neonatal    918.762704   
                                  2022 1_to_4           836.355490   
                                       early_neona

In [17]:
summary_for_cause(ylds, 'neural_tube_defects', groupby_cols)

count       mean  \
scenario                          year age_group                          
baseline                          2020 1_to_4           25.0  29.535715   
                                       early_neonatal   25.0   0.388258   
                                       late_neonatal    25.0   1.035753   
                                       post_neonatal    25.0  10.911390   
                                  2021 1_to_4           25.0  61.511151   
                                       early_neonatal   25.0   0.771514   
                                       late_neonatal    25.0   2.080997   
                                       post_neonatal    25.0  29.601197   
                                  2022 1_to_4           25.0  73.159132   
                                       early_neonatal   25.0   0.766744   
                                       late_neonatal    25.0   2.073980   
                                       post_neonatal    25.0  31.169032   
                                  2023 1_to_4           25.0  86.056052   
                                       early_neonatal   25.0   0.777569   
                                       late_neonatal    25.0   2.095764   
                                       post_neonatal    25.0  30.596274   
                                  2024 1_to_4           25.0  97.728396   
                                       early_neonatal   25.0   0.758939   
                                       late_neonatal    25.0   2.044181   
                                       post_neonatal    25.0  30.361595   
folic_acid_fortification_scale_up 2020 1_to_4           25.0  29.535715   
                                       early_neonatal   25.0   0.360517   
                                       late_neonatal    25.0   0.966928   
                                       post_neonatal    25.0  10.666904   
                                  2021 1_to_4           25.0  61.216593   
                                       early_neonatal   25.0   0.714852   
                                       late_neonatal    25.0   1.927430   
                                       post_neonatal    25.0  27.598032   
                                  2022 1_to_4           25.0  70.916499   
                                       early_neonatal   25.0   0.715837   
                                       late_neonatal    25.0   1.936732   
                                       post_neonatal    25.0  29.045431   
                                  2023 1_to_4           25.0  81.865212   
                                       early_neonatal   25.0   0.723164   
                                       late_neonatal    25.0   1.945042   
                                       post_neonatal    25.0  28.425476   
                                  2024 1_to_4           25.0  91.797626   
                                       early_neonatal   25.0   0.704051   
                                       late_neonatal    25.0   1.899605   
                                       post_neonatal    25.0  28.123012   

                                                             std        min  \
scenario                          year age_group                              
baseline                          2020 1_to_4           5.415171  21.286813   
                                       early_neonatal   0.078582   0.255471   
                                       late_neonatal    0.207543   0.694986   
                                       post_neonatal    2.118764   7.997003   
                                  2021 1_to_4          11.248537  44.863161   
                                       early_neonatal   0.146479   0.493382   
                                       late_neonatal    0.385501   1.314151   
                                       post_neonatal    5.222020  20.368658   
                                  2022 1_to_4          13.026611  49.187864   
                                   

## Load person time and try to compute some rates

In [18]:
person_time = data[('nigeria', 'person_time')]
person_time.head()

,year,age_group,sex,fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,false,person_time,21,baseline,0.0
1,2020,1_to_4,female,false,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,false,person_time,29,baseline,0.0
3,2020,1_to_4,female,false,person_time,29,folic_acid_fortification_scale_up,0.0
4,2020,1_to_4,female,false,person_time,55,baseline,0.0


In [19]:
person_time.input_draw.nunique()

25

In [20]:
index_columns = ['year', 'age_group', 'sex', 'input_draw', 'scenario']
summed_pt = person_time.groupby(index_columns).value.sum().reset_index()
summed_pt.head()

,year,age_group,sex,input_draw,scenario,value
0,2020,1_to_4,female,21,baseline,79094.679865
1,2020,1_to_4,female,21,folic_acid_fortification_scale_up,79094.679865
2,2020,1_to_4,female,29,baseline,78919.836742
3,2020,1_to_4,female,29,folic_acid_fortification_scale_up,78919.836742
4,2020,1_to_4,female,55,baseline,79070.200691


In [21]:
summed_pt.query('year == "2021"').head()

,year,age_group,sex,input_draw,scenario,value
400,2021,1_to_4,female,21,baseline,158693.529307
401,2021,1_to_4,female,21,folic_acid_fortification_scale_up,158693.532045
402,2021,1_to_4,female,29,baseline,158671.005431
403,2021,1_to_4,female,29,folic_acid_fortification_scale_up,158671.482521
404,2021,1_to_4,female,55,baseline,158238.324268


In [38]:
def compute_rate(count_data, person_time, multiplier=100_000):
    index_cols = ['input_draw', 'scenario']
    strata = ['year', 'age_group', 'sex']
    groupby_cols = strata + index_cols
    
    broadcast_cols = ['cause']
    
    count_data = count_data.groupby(groupby_cols + broadcast_cols).value.sum()
    person_time = person_time.groupby(groupby_cols).value.sum()
    
    rate = multiplier * count_data / person_time
    
    return rate.reset_index()

def compute_rate_or_ratio(numerator, denominator,
                          numerator_strata, denominator_strata,
                          multiplier=1
                         ):
    index_cols = ['input_draw', 'scenario']
    
    # The numerator has to be a subset of the denominator
#     numerator_strata = list(set(numerator_strata).union(denominator_strata))
#     print(numerator_strata)

    broadcast_cols = 
    
    numerator = numerator.groupby(index_cols+numerator_strata).value.sum()
    denominator = denominator.groupby(index_cols+denominator_strata).value.sum()
    
    rate_or_ratio = multiplier * numerator / denominator
    
    return rate_or_ratio.reset_index()

In [30]:
mortality_rate = compute_rate(deaths, person_time)
mortality_rate.head()

,year,age_group,sex,input_draw,scenario,cause,value
0,2020,1_to_4,female,21,baseline,diarrheal_diseases,132.752292
1,2020,1_to_4,female,21,baseline,lower_respiratory_infections,152.981212
2,2020,1_to_4,female,21,baseline,measles,37.929226
3,2020,1_to_4,female,21,baseline,neural_tube_defects,11.378768
4,2020,1_to_4,female,21,baseline,other_causes,557.559624


In [31]:
mortality_rate.query('value != value')

,year,age_group,sex,input_draw,scenario,cause,value


## Test my general division function to see if it gives the same result as the simpler rate function

### AAaarrrggghh!!! Indexing!!!!!

The answer is yes, but you have to re-index one of the dataframes by the index of the other in order to compare them because things ended up in different orders.

In [46]:
strata = ['year', 'age_group', 'sex']
mortality_rate2 = compute_rate_or_ratio(deaths, person_time,
                                        strata+['cause'], strata,
                                       100_000)
print(mortality_rate2.equals(mortality_rate[mortality_rate2.columns]))
mortality_rate2.head()

False


,input_draw,scenario,year,age_group,sex,cause,value
0,21,baseline,2020,1_to_4,female,diarrheal_diseases,132.752292
1,21,baseline,2020,1_to_4,female,lower_respiratory_infections,152.981212
2,21,baseline,2020,1_to_4,female,measles,37.929226
3,21,baseline,2020,1_to_4,female,neural_tube_defects,11.378768
4,21,baseline,2020,1_to_4,female,other_causes,557.559624


In [45]:
print(mortality_rate.shape)
print(mortality_rate2.shape)

(10000, 7)
(10000, 7)


In [47]:
index_cols = ['year', 'age_group', 'sex', 'input_draw', 'scenario', 'cause']
mortality_rate2.set_index(index_cols).equals(
    mortality_rate.set_index(index_cols)
)

False

In [61]:
mortality_rate2.set_index(index_cols).head(10)

value
year age_group sex    input_draw scenario cause                                   
2020 1_to_4    female 21         baseline diarrheal_diseases            132.752292
                                          lower_respiratory_infections  152.981212
                                          measles                        37.929226
                                          neural_tube_defects            11.378768
                                          other_causes                  557.559624
               male   21         baseline diarrheal_diseases            161.174643
                                          lower_respiratory_infections  183.664128
                                          measles                        19.990653
                                          neural_tube_defects             1.249416
                                          other_causes                  635.952660

In [62]:
mortality_rate.set_index(index_cols).head(10)

value
year age_group sex    input_draw scenario                          cause                                   
2020 1_to_4    female 21         baseline                          diarrheal_diseases            132.752292
                                                                   lower_respiratory_infections  152.981212
                                                                   measles                        37.929226
                                                                   neural_tube_defects            11.378768
                                                                   other_causes                  557.559624
                                 folic_acid_fortification_scale_up diarrheal_diseases            132.752292
                                                                   lower_respiratory_infections  152.981212
                                                                   measles                        37.929226
                                                                   neural_tube_defects            11.378768
                                                                   other_causes                  557.559624

In [53]:
mortality_rate2.set_index(index_cols).tail() == mortality_rate.set_index(index_cols).tail()

value
year age_group     sex  input_draw scenario                          cause                              
2024 post_neonatal male 946        folic_acid_fortification_scale_up diarrheal_diseases             True
                                                                     lower_respiratory_infections   True
                                                                     measles                        True
                                                                     neural_tube_defects            True
                                                                     other_causes                   True

In [54]:
df1 = mortality_rate.set_index(index_cols)
df2 = mortality_rate2.set_index(index_cols)
df1.equals(df2)

False

In [60]:
df1[df1.index == df2.index]

value
year age_group     sex    input_draw scenario                          cause                                    
2020 1_to_4        female 21         baseline                          diarrheal_diseases             132.752292
                                                                       lower_respiratory_infections   152.981212
                                                                       measles                         37.929226
                                                                       neural_tube_defects             11.378768
                                                                       other_causes                   557.559624
2024 post_neonatal male   946        folic_acid_fortification_scale_up diarrheal_diseases            1099.791114
                                                                       lower_respiratory_infections   756.871836
                                                                       measles                         71.033279
                                                                       neural_tube_defects             14.696541
                                                                       other_causes                  1702.349275

In [64]:
df2.reindex(df1.index).head(10)

value
year age_group sex    input_draw scenario                          cause                                   
2020 1_to_4    female 21         baseline                          diarrheal_diseases            132.752292
                                                                   lower_respiratory_infections  152.981212
                                                                   measles                        37.929226
                                                                   neural_tube_defects            11.378768
                                                                   other_causes                  557.559624
                                 folic_acid_fortification_scale_up diarrheal_diseases            132.752292
                                                                   lower_respiratory_infections  152.981212
                                                                   measles                        37.929226
                                                                   neural_tube_defects            11.378768
                                                                   other_causes                  557.559624

In [65]:
df2.reindex(df1.index).equals(df1)

True

## Compute averted deaths

In [25]:
def compute_averted(df):
    baseline = df.query('scenario == "baseline"')
    intervention = df.query('scenario != "baseline"')
    
    index_columns = list(set(baseline.columns) - set(['scenario', 'value']))
    print(index_columns)
    
    baseline = baseline.set_index(index_columns).value
    intervention = intervention.set_index(index_columns).value
    
    averted = baseline - intervention
    
    return averted.reset_index()

In [26]:
averted_deaths = compute_averted(mortality_rate)
averted_deaths.head()

['cause', 'year', 'sex', 'input_draw', 'age_group']


,cause,year,sex,input_draw,age_group,value
0,diarrheal_diseases,2020,female,21,1_to_4,2.842171e-14
1,lower_respiratory_infections,2020,female,21,1_to_4,2.842171e-14
2,measles,2020,female,21,1_to_4,7.105427e-15
3,neural_tube_defects,2020,female,21,1_to_4,1.776357e-15
4,other_causes,2020,female,21,1_to_4,1.136868e-13


In [27]:
df = averted_deaths.query('cause == "neural_tube_defects"')
index_columns = ['year','age_group', 'input_draw']
df = df.groupby(index_columns).value.sum()
df = df.reset_index()
df.groupby(['year', 'age_group']).value.describe().round(4)

count      mean       std     min       25%       50%  \
year age_group                                                               
2020 1_to_4           25.0   -0.0000    0.0000 -0.0000   -0.0000    0.0000   
     early_neonatal   25.0  128.6535  163.2809 -0.0000   -0.0000    0.0000   
     late_neonatal    25.0    9.2717   25.6265 -0.0000    0.0000    0.0000   
     post_neonatal    25.0    0.9984    2.4981  0.0000    0.0000    0.0001   
2021 1_to_4           25.0    0.1005    0.2350 -0.0000    0.0000    0.0000   
     early_neonatal   25.0  181.4824  143.2781 -0.0000  114.1680  119.0370   
     late_neonatal    25.0   17.1200   25.3095 -0.0001    0.0009    0.0053   
     post_neonatal    25.0    4.4752    3.5680  0.0000    2.4436    4.9746   
2022 1_to_4           25.0    0.2949    0.5379  0.0000    0.0001    0.0003   
     early_neonatal   25.0  125.2031  133.4191 -0.0000    0.0005  114.7925   
     late_neonatal    25.0    9.4011   20.5028 -0.0001    0.0000    0.0000   
     post_neonatal    25.0    4.5861    4.1654  0.0001    2.4221    4.9216   
2023 1_to_4           25.0    0.9191    0.7838  0.0002    0.5934    0.6134   
     early_neonatal   25.0  163.2125  150.7027 -0.0000    0.0087  118.0597   
     late_neonatal    25.0   12.5496   18.6717 -0.0000    0.0000    0.0034   
     post_neonatal    25.0    4.8638    3.7320  0.0001    2.4576    2.5471   
2024 1_to_4           25.0    1.5525    1.0928  0.0002    0.6098    1.1813   
     early_neonatal   25.0  203.2139  170.7524 -0.0000  113.1875  228.5561   
     late_neonatal    25.0   10.8402   17.7380 -0.0000    0.0017    0.0035   
     post_neonatal    25.0    4.4627    2.9413  0.0004    2.4606    4.8691   

                          75%       max  
year age_group                           
2020 1_to_4            0.0000    0.0000  
     early_neonatal  229.0384  464.7363  
     late_neonatal     0.0035   79.1514  
     post_neonatal     0.0004   10.0097  
2021 1_to_4            0.0000    0.6316  
     early_neonatal  232.5675  581.0859  
     late_neonatal    38.6248   78.5986  
     post_neonatal     5.0678   12.3074  
2022 1_to_4            0.6057    1.8594  
     early_neonatal  118.9237  468.4155  
     late_neonatal     0.0052   78.6327  
     post_neonatal     7.3329   14.7912  
2023 1_to_4            1.2140    3.0223  
     early_neonatal  237.8154  582.9674  
     late_neonatal    38.6128   40.2561  
     post_neonatal     7.4714   12.3401  
2024 1_to_4            2.3860    4.7702  
     early_neonatal  236.3720  693.9701  
     late_neonatal    38.0989   39.9107  
     post_neonatal     7.4494    9.7781

In [68]:
l1 = [1,3,4]
l1.index

<function list.index>

In [69]:
l1.index(1)

0

In [71]:
l1.index(4)

2